<a href="https://colab.research.google.com/github/ACRainieri/pyspark/blob/main/Projeto_WEB_DSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q imdb-sqlite
!pip install -q pycountry

In [ ]:
import re
import time
import sqlite3
import pycountry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from sklearn.feature_extraction.text import CountVectorizer
import warnings 

warnings.filterwarnings('ignore')
sns.set_theme(style= 'whitegrid')


In [ ]:
%%time
!imdb-sqlite

In [ ]:
conn = sqlite3.connect('imdb.db')

In [ ]:
tabelas = pd.read_sql_query("SELECT NAME as 'Table_Name' FROM sqlite_master WHERE type = 'table'", conn)

tabelas.head()  
tabelas.Table_Name


In [ ]:
tabelas = tabelas["Table_Name"].values.tolist()
tabelas

['people', 'titles', 'akas', 'crew', 'episodes', 'ratings']

In [ ]:
for tabela in tabelas:
  #consulta = "PRAGMA_TABLE_INFO({})".format(tabela)
  consulta = 'PRAGMA table_info(' + tabela +')'
  resultado = pd.read_sql_query(consulta, conn)
  print("Esquema da Tabela: "+ tabela)
  display(resultado)
 

Ajustando um Dataset, agrupando em outros os valores menores de cinco porcento o restante da lista.

In [ ]:
consulta1 = '''SELECT type, COUNT(*) as COUNT FROM titles group by type'''
resultado1 = pd.read_sql_query(consulta1, conn)

resultado1['percentual'] = (resultado1['COUNT'] / resultado1['COUNT'].sum()) * 100

others = {}

others['COUNT'] = resultado1[resultado1['percentual'] < 5] ['percentual'].sum()

others['type'] = 'others'

#display(resultado1)
display(others)

#filtra o dataframe de resultado
resultado1 = resultado1[resultado1['percentual'] > 5]

display(resultado1)

resultado1 = resultado1.append(others, ignore_index=True)

labels = [str(resultado1['type'][i]) + ' '+'['+ str(round(resultado1['percentual'][i], 2)) + '%'+']' for i in resultado1.index]

#Mapa de cores
cs = cm.Set3(np.arange(100))

#Criar figura
f = plt.figure()

plt.pie(resultado1['COUNT'], labeldistance=1, radius=3, colors=cs, wedgeprops=dict(width=0.8))
plt.legend(labels = labels, loc = 'center', prop = {'size':12})


Calculando o numero de titulos por Generos:

In [ ]:
sql_consulta_genero = "SELECT genres, COUNT(*) FROM titles WHERE type = 'movie' GROUP BY genres"

resultado2 = pd.read_sql_query(sql_consulta_genero, conn)

display(resultado2)


In [ ]:
resultado2['genres'] = resultado2['genres'].str.lower().values

#Exclui valores sem genero
temp = resultado2['genres'].dropna()

display(temp)

padrao = '(?u)\\b[\\w-]+\\b'

vetor = CountVectorizer(token_pattern = padrao, analyzer = 'word').fit(temp)

bag_generos = vetor.transform(temp)

generos_unicos = vetor.get_feature_names()